In [ ]:
# Importing libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings as wr
from pathlib import Path
from scipy.stats import pearsonr
wr.filterwarnings('ignore')
work_dir = Path.cwd().parent.parent / 'Work'
df = pd.read_excel(work_dir / "summary_tests.xlsx")
#print(df)

In [4]:
df

,Unnamed: 0,T258_dH+_mean,T258_dH+_std,T258_dH+_min,T258_dH+_25%,T258_dH+_50%,T258_dH+_75%,T258_dH+_max,T258_dH-_mean,T258_dH-_std,...,T278_SMR_diff_min,T278_SMR_diff_25%,T278_SMR_diff_50%,T278_SMR_diff_75%,T278_SMR_diff_max,T278_SMR_corr,T278_SMR_tau,id_sample,age,sex
0,0,2.923077,1.605280,1,2.00,2.0,4.00,5,1.769231,1.739437,...,-0.854969,-0.084948,-0.039704,0.080041,0.382190,-0.456972,-0.266667,0,10.219178,1
1,1,4.000000,2.645751,1,2.00,3.0,5.00,10,2.230769,2.521701,...,-0.737864,-0.101190,-0.002004,0.089127,0.214953,-0.144567,0.015385,1,11.331507,0
2,2,3.000000,1.414214,1,2.00,3.0,4.00,5,0.769231,2.773501,...,-0.312871,-0.113636,-0.029562,0.008153,0.397260,-0.599937,-0.450188,2,11.465753,0
3,3,2.846154,1.772294,1,2.00,2.0,4.00,7,1.538462,2.875627,...,-0.860465,-0.154362,-0.041209,0.100264,0.459375,-0.266544,-0.130703,3,11.701370,0
4,4,4.384615,2.063107,2,2.00,4.0,6.00,8,0.846154,1.214232,...,-0.913232,-0.115234,-0.018072,0.052930,0.500554,-0.184131,-0.103854,4,12.304110,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3219,3219,15.538462,10.063261,7,9.00,12.0,19.00,42,5.307692,3.326275,...,-0.261944,-0.103728,-0.051200,0.083958,0.212740,-0.194597,-0.159420,3219,83.567123,1
3220,3220,4.692308,2.015962,2,3.00,5.0,6.00,8,2.153846,1.405119,...,-1.381188,-0.099881,-0.014825,0.039242,0.610707,-0.118538,-0.200618,3220,84.380822,0
3221,3221,12.916667,5.350588,6,8.75,12.0,15.25,23,1.500000,1.834022,...,-0.784203,-0.134588,0.049347,0.110667,0.220359,-0.186524,-0.166667,3221,86.531507,0
3222,3222,7.384615,4.213836,2,5.00,6.0,8.00,17,2.461538,2.787334,...,-0.373239,-0.112847,-0.003512,0.052903,0.303716,-0.474331,-0.356436,3222,87.306849,1


In [2]:
bins = [0, 30, 55, 100]
labels = ['<30', '30-55', '56+']
df['age_group'] = pd.cut(df['age'], bins=bins, labels=labels, right=False)

#df.to_excel("D:/Work/summary_tests_with_age_group.xlsx", index=False)
delete_columns = ['id_sample', 'age', 'sex', 'Unnamed: 0', 'age_group']

features = [column for column in df.columns if column not in delete_columns]

df.loc[df["sex"] == 0, "sex"] = 'М'
df.loc[df["sex"] == 1, "sex"] = 'Ж'

save_dir = work_dir / 'Box_plots_for_features'
save_dir.mkdir(parents=True, exist_ok=True)

def is_dir_not_empty(dir):
    return any(dir.iterdir())

def delete_files_in_dir(dir):
    for file in dir.glob('*'):
        if file.is_file():
            file.unlink() 

def get_quantile(dataFrame, column, percent):
    return dataFrame[column].quantile(percent)

def BoxPlotsForFeatures(data, feature, feature_name, path_to_file):
    
      ax = plt.subplots(figsize=(10, 6), dpi=300)
      ax = sns.boxplot(
            data=data, 
            x='age_group', 
            y=feature, 
            hue='sex', 
            palette={'М': "skyblue", 'Ж': "lightcoral"}, 
            hue_order = ['М', 'Ж']
      )
      y_max = get_quantile(df, feature, 0.995)
      y_min = get_quantile(df, feature, 0.001)
      ax.set_ylim(ymax=y_max, ymin=y_min)
      ax.set_title(f'Диаграмма размаха для признака: {feature_name}', fontsize=14)
      ax.set_xlabel('Возраст')
      ax.set_ylabel(feature_name)
      plt.xticks(fontsize=12)
      plt.yticks(fontsize=12)
      ax.legend(title='Пол', loc='upper right')
      # ax.text(0.015, 0.95, f'Признак: {feature}', 
      #       transform=ax.transAxes,
      #       color='red',
      #       fontsize=14
      # )


      path = path_to_file / f'{feature}.png'
      plt.savefig(path, bbox_inches='tight')
      plt.close()


if (is_dir_not_empty(save_dir)):
        delete_files_in_dir(save_dir)

for feature in features:
    splitted_f = feature.split('_', 2)
    feature_name = ' '.join(splitted_f)
    
    BoxPlotsForFeatures(df, feature, feature_name, save_dir)